In [1]:
import pandas as pd
import numpy as np

/Users/vega7unk/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import os
print(os.getcwd())

/Users/vega7unk/Documents/4th Sem DSI/ilab project/ilab-group-12-1-fall-detection/notebooks


In [3]:
pd.set_option('display.max_columns', None)

df = pd.read_pickle("../data/raw/df_resample_100ms.pkl")

In [4]:
df.head()

,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,label,subject_id,trial,age,height,weight,gender
timestamp,,,,,,,,,,,,,
1970-01-01 00:38:38.700,0.948777,-9.636166,0.002699,0.003818,0.016875,0.006643,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:38.800,0.948993,-9.759188,0.087482,0.020693,0.027565,0.015669,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:38.900,1.071514,-9.787465,-0.093610,0.168721,0.075747,0.037797,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:39.000,1.135679,-9.754036,0.065878,0.339488,0.078970,0.040134,BSC,1.0,1.0,32.0,180.0,85.0,M
1970-01-01 00:38:39.100,1.126174,-9.443248,0.075006,0.594494,-0.012065,0.103557,BSC,1.0,1.0,32.0,180.0,85.0,M


In [5]:
df['subject_id'].unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.,
       14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26.,
       27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.,
       40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50., 51., 52.,
       53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65.,
       66., 67.])

In [6]:
data = df.copy()

In [7]:
data.drop(['trial','age','height','weight','gender'], axis=1, inplace=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 846950 entries, 1970-01-01 00:38:38.700000 to 1970-01-01 04:49:55.600000
Data columns (total 8 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   acc_x       846950 non-null  float64
 1   acc_y       846950 non-null  float64
 2   acc_z       846950 non-null  float64
 3   gyro_x      846950 non-null  float64
 4   gyro_y      846950 non-null  float64
 5   gyro_z      846950 non-null  float64
 6   label       846950 non-null  object 
 7   subject_id  846950 non-null  float64
dtypes: float64(7), object(1)
memory usage: 58.2+ MB


In [9]:
data['subject_id'].nunique()

67

In [10]:
def create_sliding_windows(df, window_size, step_size):
    X = []  # input features
    y = []  # labels
    
    # Loop over the DataFrame with step size
    for start_idx in range(0, len(df) - window_size + 1, step_size):
        end_idx = start_idx + window_size
        
        window = df.iloc[start_idx:end_idx]
        
        # Drop label and other non-signal columns for features
        features = window[["acc_x", "acc_y", "acc_z", "gyro_x", "gyro_y", "gyro_z"]].values
        
        # Take last label in the window
        label = window["label"].iloc[-1]
        
        X.append(features)
        y.append(label)
        
    return np.array(X), np.array(y)

In [11]:
df_resample_100ms = data.copy()

# Parameters
window_duration_seconds = 2.56
sampling_rate = 10  # Hz
window_size = 26
step_size = window_size // 2  # 50% overlap

X_windows, y_windows = create_sliding_windows(df_resample_100ms, window_size, step_size)

print(f"Shape of X: {X_windows.shape}")  # (num_windows, 26, 6)
print(f"Shape of y: {y_windows.shape}")  # (num_windows,)

Shape of X: (65149, 26, 6)
Shape of y: (65149,)


In [13]:
print(df_resample_100ms.isnull().values.any())

False
